In [21]:
import concurrent.futures
from multiprocessing import Lock, cpu_count
import time
import hashlib
import string
import itertools

In [2]:
import matplotlib.pyplot as plt
import plotly.graph_objs as go

#### VISUALIZE DATA

In [3]:
def plt_visualize(dataset):
    
    # LABELS
    labels = [x for x in range(len(dataset))]

    plt.figure(figsize=(16, 6))
    plt.plot(labels, dataset, linestyle='-', marker='')
        
    plt.grid(True)
    plt.show()

In [4]:
def plotly_g(dataset):
    
    # CREATE LABELS
    labels = [x for x in range(len(dataset))]

    # PLOT CONTAINER
    structure = []
    
    # LINE COLOURS
    colors = ['#e75f5b', '#52af52', '#403638', '#a93581', '#005073', '#f2ae42']
    
    # APPEND DOT CHART
    structure.append(go.Scatter(
        mode='line',
        x=labels,
        y=dataset,
        line=dict(width=1),
        marker=dict(color=colors[0]),
        opacity=0.6,
        yaxis='y2'
    ))
        

    # LAYOUT PARAMS
    layout = go.Layout(
        yaxis = dict(domain = [0, 0.2],
        showticklabels=False),
        margin=dict(l=20, r=20, t=20, b=20)
    )
    
    # CREATE THE FIGURE
    fig = go.Figure(
        data=structure,
        layout=layout
    )
    
    # FINALLY SHOW THE GRAPH
    fig.show()

#### TIMER

In [5]:
class create_timer():
    
    # START TIMER
    def __init__(self, name):
        self.start = time.perf_counter()
        
    # FINISH TIMER
    def finish(self):
        self.end = time.perf_counter()
        self.difference()

    # PRINT DIFFERENCE
    def difference(self):
        self.delta = round(self.end-self.start, 3)

In [6]:
def parse_results(results):
    
    # HELPER VARS
    labels = []
    summa = 0
    length = 0
    
    # COMPUTE SUM
    for timestamp, delta in results:
        
        # PUSH TO CONTAINERS
        labels.append(timestamp)
        
        summa += delta
        length += 1
        
    # COMPUTE AVG
    avg = summa / length
    
    return round(avg, 3), labels

#### RUNTIME WRAPPER

In [7]:
def wrapper(func, params, parallel=False):
    
    # USE MULTI-THREADING BY DEFAULT
    method = concurrent.futures.ThreadPoolExecutor
    
    # USE PARALLEL PROCESSING WHEN REQUESTED
    if parallel:
        method = concurrent.futures.ProcessPoolExecutor
        
    # CREATE MUTEX
    # mutex = Lock()
    # mutex.acquire()
    # mutex.release()
    
    # START TIMER
    timer = create_timer('WRAPPER')
    
    # EXECUTE FUNCS
    with method() as executor:
        results = executor.map(func, params)

    # END TIMER
    timer.finish()
    
    # AVG TIME PER PROCESS
    avg = timer.delta / len(params)
    
    show(timer.delta, avg)

In [8]:
def baseline(func, params):
    
    # START TIMER
    timer = create_timer('BASELINE')

    # RUN EACH PARAM
    for value in params:
        block = func(value)

    # END TIMER
    timer.finish()
    
    # AVG TIME PER PROCESS
    avg = timer.delta / len(params)
    
    show(timer.delta, avg)

In [9]:
def show(total, avg):
    print('TOTAL:\t', round(total, 3))
    print('AVG:\t', round(avg, 3))

#### SHARED PROCESSES

In [10]:
def delayed_func(input_string):
    
    # ENCODE AND HASH
    encoded = input_string.encode('utf-8')
    hashed = hashlib.sha256(encoded).hexdigest()
    
    time.sleep(0.01)

    #if hashed == 'foo':
    #    print('match found:', hashed)

In [11]:
def hash_recurs(input_string, n_times=4000):
    
    # FORMAT
    encoder_format = 'utf-8'
    
    # ENCODE INITIAL DATA
    output = input_string.encode(encoder_format)
    
    # HASH RECURSIVELY
    for _ in range(n_times):
        output = hashlib.sha256(output).hexdigest().encode(encoder_format)
        
    return output.decode(encoder_format)

#### PASSWORD PERMUTATIONS

In [12]:
def generate_passwords(charset, length):

    # CONTAINER
    container = []
    
    # LOOP THROUGH EACH PERMUTATION
    for combination in itertools.product(*[charset] * length):  
        word = ''.join(combination)
        container.append(word)
        
    return container

In [13]:
permutations = generate_passwords(
    charset=string.digits + string.ascii_lowercase,
    length=2
)

#### PROCESS WITH ARTIFICIAL DELAY -- THREADING BOUND?

In [14]:
baseline(
    func=delayed_func,
    params=permutations
)

TOTAL:	 13.152
AVG:	 0.01


In [15]:
wrapper(
    func=delayed_func,
    params=permutations,
    parallel=True
)

TOTAL:	 1.782
AVG:	 0.001


In [16]:
wrapper(
    func=delayed_func,
    params=permutations,
    parallel=False
)

TOTAL:	 1.118
AVG:	 0.001


#### PROCESS WITH RECURSIVE HASHING -- CPU BOUND

In [17]:
baseline(
    func=hash_recurs,
    params=permutations
)

TOTAL:	 3.858
AVG:	 0.003


In [18]:
wrapper(
    func=hash_recurs,
    params=permutations,
    parallel=True
)

TOTAL:	 0.606
AVG:	 0.0


In [20]:
wrapper(
    func=hash_recurs,
    params=permutations,
    parallel=False
)

TOTAL:	 6.471
AVG:	 0.005
